In [1]:
import sys
import os
import argparse

# 添加项目路径
sys.path.append('/Users/haha/Story')  # 你的项目根目录

# 导入所有必要模块
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_sentence, compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import analyze_dialogue_insertions, run_dialogue_insertion, analyze_dialogue_insertions_v2
from src.utils.utils import extract_behavior_llm, convert_dialogue_dict_to_list
from src.sync.plot_sync_manager import sync_plot_and_dialogue_from_behavior
from src.sync.auto_propagate_plot_update import auto_propagate_plot_update
from src.analysis.character_state_tracker import run_character_state_tracker
from src.utils.logger import append_log, build_log_record, build_simple_log, init_log_path
from src.version_namer import build_version_name 

print("✅ 所有模块导入成功")

#

✅ 所有模块导入成功


In [2]:
# 置参数
topic = "小红帽"
style = "科幻改写"
reorder_mode = "linear"
use_cache = False
behavior_model = "gpt-4.1"
temperature = 0.7
seed = 1

print(f"📋 测试参数:")
print(f"   Topic: {topic}")
print(f"   Style: {style}")
print(f"   Reorder mode: {reorder_mode}")
print(f"   Temperature: {temperature}")
print(f"   Seed: {seed}")

📋 测试参数:
   Topic: 小红帽
   Style: 科幻改写
   Reorder mode: linear
   Temperature: 0.7
   Seed: 1


In [3]:
# 自动构建版本名称
version = build_version_name(
    topic=topic,
    style=style,
    temperature=temperature,
    seed=seed,
    order_mode=reorder_mode
)

print(f"🏷️ 生成版本名: {version}")

# 创建输出文件夹
def ensure_output_dir(version):
    folder = os.path.join(output_dir, version)
    os.makedirs(folder, exist_ok=True)
    return folder

def step_file(version, filename):
    return os.path.join(output_dir, version, filename)

folder = ensure_output_dir(version)
role_state = {}

plot_log_path = init_log_path(folder, "plot")
dialogue_log_path = init_log_path(folder, "dialogue")

print(f"📁 输出文件夹: {folder}")
print(f"📝 日志路径: plot={plot_log_path}, dialogue={dialogue_log_path}")

🏷️ 生成版本名: 小红帽_科幻_linear_T0.7_s1
📁 输出文件夹: /Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1
📝 日志路径: plot=/Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1/logs/plot_log.jsonl, dialogue=/Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1/logs/dialogue_log.jsonl


# Step 1 - Outline Generation

In [4]:
print("=== Step 1: Outline Generation ===")

# 复制main_pipeline.py的outline生成逻辑
outline_base_path = os.path.join(output_dir, "reference_outline", f"{topic}_{style}_T{temperature}_s{seed}outline.json")
os.makedirs(os.path.dirname(outline_base_path), exist_ok=True)

if os.path.exists(outline_base_path) and use_cache:
    outline = load_json(outline_base_path)
    print(f"📖 已加载共享 outline：{outline_base_path}")
else:
    print(f"🔄 生成新的outline...")
    outline = generate_outline(topic=topic, style=style, custom_instruction="")
    save_json(outline, "reference_outline", f"{topic}_{style}_T{temperature}_s{seed}_outline.json")
    print(f"💾 生成并保存共享 outline：{outline_base_path}")

print(f"✅ Outline生成完成")
print(f"   章节数: {len(outline)}")
for i, ch in enumerate(outline):
    print(f"   {i+1}. {ch.get('chapter_id', 'Unknown')}: {ch.get('title', 'Unknown')}")
    print(f"      Summary: {ch.get('summary', '')[:60]}...")

=== Step 1: Outline Generation ===
🔄 生成新的outline...
yuanshi content: [
  {
    "chapter_id": "Chapter 1",
    "title": "星际快递任务",
    "summary": "小红帽是一名年轻的星际快递员，接到任务要将医疗芯片送往外婆所在的边境空间站。"
  },
  {
    "chapter_id": "Chapter 2",
    "title": "穿越虫洞",
    "summary": "小红帽驾驶飞船穿越危险的虫洞，途中遇到神秘的太空风暴。"
  },
  {
    "chapter_id": "Chapter 3",
    "title": "遭遇机械狼",
    "summary": "在接近空间站时，小红帽被一只伪装成维修机器人的机械狼拦截。"
  },
  {
    "chapter_id": "Chapter 4",
    "title": "机械狼的阴谋",
    "summary": "机械狼假扮成外婆，试图窃取医疗芯片以控制空间站的主系统。"
  },
  {
    "chapter_id": "Chapter 5",
    "title": "智斗机械狼",
    "summary": "小红帽识破机械狼的伪装，利用高科技装备与其展开激烈对抗。"
  },
  {
    "chapter_id": "Chapter 6",
    "title": "拯救外婆",
    "summary": "小红帽成功救出被困的外婆，并将医疗芯片安全交付。"
  },
  {
    "chapter_id": "Chapter 7",
    "title": "归途与成长",
    "summary": "任务完成后，小红帽踏上归途，收获了成长与勇气。"
  }
]...
💾 生成并保存共享 outline：/Users/haha/Story/data/output/reference_outline/小红帽_科幻改写_T0.7_s1outline.json
✅ Outline生成完成
   章节数: 7
   1. Chapter 1: 星际快递任务
      Summary: 小红帽是一名年轻的星

# Step 2 - Chapter Reordering

In [5]:
print("=== Step 2: Chapter Reordering ===")

# 复制main_pipeline.py的章节重排逻辑
if reorder_mode == "linear":
    reorder_outline_raw = outline
    save_json(outline, version, "test_outline.json")
    print("✅ 使用 linear 顺序（直接来自 outline）")

elif reorder_mode == "nonlinear":
    save_json(outline, version, "test_outline_linear.json")
    reorder_path = os.path.join(output_dir, "reference_reorder", f"{topic}_{style}_T{temperature}_s{seed}_nonlinear.json")
    os.makedirs(os.path.dirname(reorder_path), exist_ok=True)

    if os.path.exists(reorder_path) and use_cache:
        reorder_outline_raw = load_json(reorder_path)
        print(f"📖 已加载 cached nonlinear 顺序：{reorder_path}")
    else:
        print(f"🔄 生成nonlinear顺序...")
        reorder_outline_raw = reorder_chapters(outline, mode="nonlinear")

        # 添加日志记录
        reorder_log_path = init_log_path(folder, "reorder")
        reorder_log = build_simple_log(
            module="chapter_reorder",
            task_name=version,
            input_data={"outline": outline},
            output_data={"reorder_result": reorder_outline_raw}
        )
        append_log(reorder_log_path, reorder_log)

        # 检查是否真的生成了 new_order 字段
        if not any("new_order" in ch for ch in reorder_outline_raw):
            print("⚠️ LLM 重排失败：未检测到任何 new_order 字段，回退为原始顺序")
        else:
            print("✅ reorder_chapters 成功生成非线性顺序")

        save_json(reorder_outline_raw, "reference_reorder", f"{topic}_{style}_T{temperature}_s{seed}_nonlinear.json")
        print(f"💾 生成 nonlinear 顺序并缓存：{reorder_path}")

else:
    raise ValueError("order_mode 必须为 'linear' 或 'nonlinear'")

# 统一结构：补全 summary 字段
reorder_outline = []
for reordered_ch in reorder_outline_raw:
    match = next((x for x in outline if x["chapter_id"] == reordered_ch["chapter_id"]), None)
    if match:
        merged = {
            "chapter_id": reordered_ch["chapter_id"],
            "title": reordered_ch["title"],
            "summary": match.get("summary", "")
        }
        if "new_order" in reordered_ch:
            merged["new_order"] = reordered_ch["new_order"]
        reorder_outline.append(merged)

save_json(reorder_outline, version, "test_reorder_outline.json")
print("✅ 章节顺序处理完成（已保留 summary）")

# 显示最终顺序
print(f"📋 最终章节顺序:")
for i, ch in enumerate(reorder_outline):
    order_info = f" (原顺序: {ch.get('new_order', i+1)})" if 'new_order' in ch else ""
    print(f"   {i+1}. {ch['chapter_id']}: {ch['title']}{order_info}")

=== Step 2: Chapter Reordering ===
✅ 使用 linear 顺序（直接来自 outline）
✅ 章节顺序处理完成（已保留 summary）
📋 最终章节顺序:
   1. Chapter 1: 星际快递任务
   2. Chapter 2: 穿越虫洞
   3. Chapter 3: 遭遇机械狼
   4. Chapter 4: 机械狼的阴谋
   5. Chapter 5: 智斗机械狼
   6. Chapter 6: 拯救外婆
   7. Chapter 7: 归途与成长


# Step 3 - Character Generation

In [6]:
print("=== Step 3: Character Generation ===")

character_path = step_file(version, "characters.json")
if use_cache and os.path.exists(character_path):
    characters = load_json(character_path)
    print("📖 已加载角色设定")
else:
    print(f"🔄 生成角色设定...")
    characters = generate_characters_v1(reorder_outline)
    save_json(characters, version, "characters.json")
    print("💾 生成角色设定完成")

print(f"✅ 角色生成完成")
print(f"   角色数: {len(characters)}")
for i, char in enumerate(characters):
    print(f"   {i+1}. {char.get('name', 'Unknown')}")
    print(f"      特征: {char.get('traits', 'Unknown')[:50]}...")
    print(f"      背景: {char.get('background', 'Unknown')[:50]}...")

=== Step 3: Character Generation ===
🔄 生成角色设定...
yuanshi content: [
    {
        "name": "小红帽",
        "role": "主角，星际快递员",
        "traits": "勇敢、机智、善良、富有责任感",
        "background": "年轻的星际快递员，受过专业飞船驾驶和自卫训练，家人居住在边境空间站。",
        "motivation": "完成任务，将医疗芯片安全送达外婆手中，保护家人和空间站的安全。"
    },
    {
        "name": "外婆",
        "role": "小红帽的亲人，空间站居民",
        "traits": "慈爱、坚强、智慧、身体虚弱",
        "background": "居住在边境空间站的退休科学家，因健康问题需要医疗芯片。",
        "motivation": "希望康复并继续守护空间站，同时关心小红帽的成长。"
    },
    {
        "name": "机械狼",
        "role": "反派，智能机械体",
        "traits": "狡猾、冷酷、善于伪装、计算精密",
        "background": "由黑市势力制造的高智能机械体，具备伪装和黑客能力，长期潜伏于空间站周边。",
        "motivation": "窃取医疗芯片，控制空间站主系统，为幕后势力谋取利益。"
    },
    {
        "name": "空间站站长",
        "role": "空间站管理者",
        "traits": "严谨、负责、警觉、信任下属",
        "background": "负责边境空间站的日常运作和安全，熟悉站内所有人员和设备。",
        "motivation": "保障空间站安全，维护居民利益。"
    },
    {
        "name": "快递公司调度员",
        "role": "小红帽的上级，任务派发者",
        "traits": "高效、冷静、善于沟通、经验丰富",
   

# Step 4 - Story Expansion

In [7]:
print("=== Step 4: Story Expansion ===")

plot_path = step_file(version, "story.json")
if use_cache and os.path.exists(plot_path):
    story = load_json(plot_path)
    print("📖 已加载故事内容")
else:
    print(f"🔄 生成故事内容...")
    story = expand_story_v1(reorder_outline, characters, custom_instruction="")
    
    # 确保每章都有正确的ID和标题
    for idx, ch in enumerate(story):
        ch.setdefault("chapter_id", reorder_outline[idx]["chapter_id"])
        ch.setdefault("title", reorder_outline[idx]["title"])

        # 记录日志
        log = build_log_record(
            module="expand_story", step="plot",
            task_name=version, chapter_id=ch["chapter_id"],
            model=behavior_model, 
            input_data={"outline": reorder_outline[idx]},
            output_data={"plot": ch["plot"]},
            temperature=temperature, 
            seed=seed
        )
        append_log(plot_log_path, log)

    save_json(story, version, "story.json")
    print("💾 故事内容生成完成")

print(f"✅ 故事生成完成")
print(f"   章节数: {len(story)}")
for i, ch in enumerate(story):
    print(f"   {i+1}. {ch.get('chapter_id', 'Unknown')}: {ch.get('title', 'Unknown')}")
    print(f"      Plot长度: {len(ch.get('plot', ''))} 字符")
    print(f"      Scene长度: {len(ch.get('scene', ''))} 字符")
    print(f"      Plot预览: {ch.get('plot', '')[:80]}...")

=== Step 4: Story Expansion ===
🔄 生成故事内容...
📨 第 Chapter 1 章 LLM 返回片段： {\n    "scene": "边境空间站外的星际停泊区，银白色的空间站悬浮在深邃宇宙中，舷窗透出温暖灯光。停泊区内，星际快递飞船整装待发，机械臂缓缓移动，检修着各类飞行器。远处，空间站的巨大防护罩闪烁着淡蓝色光芒，偶尔有维修机器人穿梭其中，背景是无垠星海与遥远星球的轮廓。",\n    "chara
yuanshi content: {
    "scene": "边境空间站外的星际停泊区，银白色的空间站悬浮在深邃宇宙中，舷窗透出温暖灯光。停泊区内，星际快递飞船整装待发，机械臂缓缓移动，检修着各类飞行器。远处，空间站的巨大防护罩闪烁着淡蓝色光芒，偶尔有维修机器人穿梭其中，背景是无垠星海与遥远星球的轮廓。",
    "characters": ["小红帽", "快递公司调度员"],
    "plot": "小红帽身穿红色快递制服，背着装有医疗芯片的特制包裹，站在自己的快递飞船旁，正通过头戴通讯器与快递公司调度员进行最后的任务确认。调度员的影像投射在飞船舱门旁的全息屏幕上，神情专注地叮嘱小红帽注意途中安全，尤其警惕近期频发的黑市机械体活动。小红帽坚定地点头，目光中透出责任感。飞船舱门缓缓关闭，动力系统启动，舱内灯光亮起，准备启程前往空间站内部，将医疗芯片安全送达外婆手中。远处的维修机器人“小蓝”挥动机械臂，向小红帽致意，空间站的安保队员在巡逻，整个场景充满紧张而有序的气氛。"
}...
📨 原始 LLM 返回内容：
{
    "scene": "边境空间站外的星际停泊区，银白色的空间站悬浮在深邃宇宙中，舷窗透出温暖灯光。停泊区内，星际快递飞船整装待发，机械臂缓缓移动，检修着各类飞行器。远处，空间站的巨大防护罩闪烁着淡蓝色光芒，偶尔有维修机器人穿梭其中，背景是无垠星海与遥远星球的轮廓。",
    "characters": ["小红帽", "快递公司调度员"],
    "plot": "小红帽身穿红色快递制服，背着装有医疗芯片的特制包裹，站在自己的快递飞船旁，正通过头戴通讯器与快递公司调度员进行最后的任务确认。调度员的影像投射在飞船舱门旁的全息屏幕上，神情专注地叮嘱小红帽注意途中安全，尤其警惕近期频发的黑市机械体活动。小

# Step 5&6 - 新版对话生成（关键测试）

In [8]:
print("=== Step 5&6: 新版对话生成（句子级分析 + 章节级兼容） ===")

print(f"🔄 开始分析 {len(story)} 个章节...")

try:
    # 调用新的v2函数
    chapter_results, sentence_results, behavior_timeline = analyze_dialogue_insertions_v2(story, characters)
    
    print(f"✅ 对话分析完成！")
    print(f"   Chapter results: {len(chapter_results)} 个章节")
    print(f"   Sentence results: {len(sentence_results)} 个句子") 
    print(f"   Behavior timeline: {len(behavior_timeline)} 条记录")
    
    # 保存三种格式的数据
    save_json(chapter_results, version, "dialogue_marks.json")        # 兼容格式
    save_json(sentence_results, version, "sentence_dialogues.json")    # 句子级详细分析
    save_json(behavior_timeline, version, "behavior_timeline_raw.json")  # 原始behavior数据
    
    # 设置dialogue_result以保持后续流程兼容
    dialogue_result = chapter_results
    print("💾 三种格式数据已保存")
    
    # 快速统计
    print(f"\n📊 句子分析统计:")
    chapter_stats = {}
    dialogue_count = 0
    
    for sentence in sentence_results:
        chapter_id = sentence['chapter_id']
        if chapter_id not in chapter_stats:
            chapter_stats[chapter_id] = {'total': 0, 'dialogue': 0}
        chapter_stats[chapter_id]['total'] += 1
        if sentence['need_to_action'] == 1:
            chapter_stats[chapter_id]['dialogue'] += 1
            dialogue_count += 1
    
    print(f"   总句子数: {len(sentence_results)}")
    print(f"   需要对话的句子: {dialogue_count}")
    
    for chapter_id, stats in chapter_stats.items():
        print(f"   {chapter_id}: {stats['total']} 句，{stats['dialogue']} 句需对话")
    
    # Behavior统计
    if behavior_timeline:
        behavior_chars = {}
        for behavior in behavior_timeline:
            char = behavior['character']
            if char not in behavior_chars:
                behavior_chars[char] = 0
            behavior_chars[char] += 1
        
        print(f"\n👥 Behavior统计:")
        for char, count in behavior_chars.items():
            print(f"   {char}: {count} 个行为")
    
    print(f"✅ 新版对话生成完成")

except Exception as e:
    print(f"❌ 对话生成失败: {e}")
    import traceback
    traceback.print_exc()
    # 如果失败，可以在这里停止或使用旧方法

=== Step 5&6: 新版对话生成（句子级分析 + 章节级兼容） ===
🔄 开始分析 7 个章节...
章节Chapter 1分割为5个句子
yuanshi content: [
    {
        "sentence": "小红帽身穿红色快递制服,背着装有医疗芯片的特制包裹,站在自己的快递飞船旁,正通过头戴通讯器与快递公司调度员进行最后的任务确认.",
        "need_to_action": 1,
        "actor_list": ["小红帽", "快递公司调度员"]
    },
    {
        "sentence": "调度员的影像投射在飞船舱门旁的全息屏幕上,神情专注地叮嘱小红帽注意途中安全,尤其警惕近期频发的黑市机械体活动.",
        "need_to_action": 1,
        "actor_list": ["快递公司调度员", "小红帽"]
    },
    {
        "sentence": "小红帽坚定地点头,目光中透出责任感.",
        "need_to_action": 0,
        "actor_list": ["小红帽"]
    },
    {
        "sentence": "飞船舱门缓缓关闭,动力系统启动,舱内灯光亮起,准备启程前往空间站内部,将医疗芯片安全送达外婆手中.",
        "need_to_action": 0,
        "actor_list": ["小红帽"]
    },
    {
        "sentence": "远处的维修机器人“小蓝”挥动机械臂,向小红帽致意,空间站的安保队员在巡逻,整个场景充满紧张而有序的气氛.",
        "need_to_action": 1,
        "actor_list": ["维修机器人“小蓝”", "小红帽"]
    }
]...

🔍 开始生成对话，候选角色: ['小红帽', '快递公司调度员']
  📤 发送第一个prompt给小红帽
  📥 LLM原始返回: {"dialogue": "调度员，我已经到达指定坐标，医疗芯片包裹状态良好，请确认下一步投递指令。", "action": "调整头戴通讯器，检查包裹固定情况，等

In [9]:
# 在 Step 5&6 之后添加
print("\n=== 验证句子级对话数据 ===")

# 检查保存的文件
sentence_dialogues_path = os.path.join(folder, "sentence_dialogues.json")
if os.path.exists(sentence_dialogues_path):
    saved_sentence_data = load_json(sentence_dialogues_path)
    print(f"✅ sentence_dialogues.json 存在，包含 {len(saved_sentence_data)} 个句子")
    
    # 检查是否有dialogue和action
    dialogues_with_action = 0
    for sentence in saved_sentence_data:
        if sentence.get("dialogue"):
            for d in sentence["dialogue"]:
                if d.get("action"):
                    dialogues_with_action += 1
                    print(f"   找到action: {d['speaker']}: {d['action']}")
                    break  # 只打印第一个作为示例
    
    print(f"📊 有 {dialogues_with_action} 个句子包含带action的对话")
else:
    print("❌ sentence_dialogues.json 不存在！")


=== 验证句子级对话数据 ===
✅ sentence_dialogues.json 存在，包含 42 个句子
   找到action: 小红帽: 调整头戴通讯器,检查包裹固定情况,等待调度员回复
   找到action: 快递公司调度员: 全息屏幕上的影像微微闪烁,调度员目光严肃地看向小红帽,语气中带着关切和警觉.
   找到action: 维修机器人“小蓝”: 挥动机械臂,朝小红帽友好地打招呼
   找到action: 快递公司调度员: 调度员在全息影像中调出虫洞穿越风险分析图,指向关键注意事项.
   找到action: 机械狼: 机械狼缓缓逼近,金属爪子在地面上划出刺耳的声音,红色电子眼闪烁着危险的光芒.
   找到action: 机械狼: 机械狼用变声器发出低沉的威胁,同时在终端上快速敲击代码,锁死控制室所有出口.
   找到action: 小红帽: 小红帽通过随身终端发送静音信息,环顾四周,警惕地观察环境.
   找到action: 机械狼: 机械狼从阴影中缓缓走出,举起手臂上的扫描仪,试图取得小红帽的信任.
   找到action: 小红帽: 小红帽一边说话,一边悄悄按下走廊上的紧急联络终端,同时慢慢向维修通道靠近,准备引诱机械狼跟过来.
   找到action: 小红帽: 小红帽快速在控制台上输入指令,切换能源核心的访问权限,同时悄悄激活了备用防火墙,为安保队长和小蓝争取更多时间.
   找到action: 小红帽: 通过通讯器呼叫维修机器人“小蓝”,请求技术支持
   找到action: 机械狼: 用温柔的外婆声音通过舱门扬声器说话,试图让小红帽放松警惕,靠近舱门.
   找到action: 小红帽: 请求小蓝启动扫描程序,对‘外婆’进行身份识别.
   找到action: 小红帽: 小红帽警惕地挡在医疗舱前,紧握手中的工具,随时准备应对机械狼的反扑.
   找到action: 外婆: 外婆微笑着伸出手,轻轻握住小红帽的手,眼中闪烁着泪光.
   找到action: 小红帽: 小红帽迅速挡在外婆前面,警惕地盯着机械狼,准备保护芯片.
   找到action: 维修机器人“小蓝”: 向安保队长发出警告信号,同时远程激活空间站防御系统
   找到action: 小红帽: 小红帽警惕地挡在外婆和芯片前,向小蓝投去感激的目光,同时严阵以待,防止机械狼做

# Step 8 - 检查长度匹配和记录日志

In [10]:
print("=== 数据一致性检查和日志记录 ===")

# 检查长度是否匹配
if len(story) != len(dialogue_result):
    print(f"⚠️ 警告：story 有 {len(story)} 章，但 dialogue_result 只有 {len(dialogue_result)} 条对白")
else:
    print(f"✅ 数据长度匹配：{len(story)} 章节")

# 记录对话生成日志
print(f"📝 记录对话生成日志...")
for ch, dlg in zip(story, dialogue_result):
    log = build_log_record(
        module="dialogue_inserter", step="dialogue",
        task_name=version, chapter_id=ch["chapter_id"],
        model=behavior_model,
        input_data={"plot": ch["plot"]},
        output_data={"dialogue": dlg["dialogue"]},
        temperature=temperature, seed=seed
    )
    append_log(dialogue_log_path, log)

print(f"✅ 日志记录完成")

=== 数据一致性检查和日志记录 ===
✅ 数据长度匹配：7 章节
📝 记录对话生成日志...
✅ 日志记录完成


# Step 6.5 - 新版Behavior处理

In [11]:
print("=== Step 6.5: 新版Behavior处理 ===")

# 组织角色弧线
character_arcs = {}
for item in behavior_timeline:
    char = item["character"]
    if char not in character_arcs:
        character_arcs[char] = []
    character_arcs[char].append({
        "chapter": item["chapter_id"],
        "sentence": item["sentence_index"],
        "behavior": item["behavior"],
        "scene": item["scene_context"][:30] + "..." if len(item["scene_context"]) > 30 else item["scene_context"]
    })

# 生成完整的behavior_trace
behavior_trace = {
    "timeline": behavior_timeline,
    "character_arcs": character_arcs,
    "statistics": {
        "total_dialogue_moments": len(behavior_timeline),
        "characters_behavior_count": {char: len(arcs) for char, arcs in character_arcs.items()}
    },
    "legacy_behaviors": [f"{item['character']}：{item['behavior']}" for item in behavior_timeline]
}

save_json(behavior_trace, version, "behavior_trace.json")

# 兼容role_state
role_state = {}
for item in behavior_timeline:
    role = item["character"]
    behavior_item = item["behavior"]
    role_state.setdefault(role, [])
    if behavior_item not in role_state[role]:
        role_state[role].append(behavior_item)

print(f"✅ 新版behavior trace生成完成")
print(f"📊 角色弧线统计:")
for char, arcs in character_arcs.items():
    print(f"   {char}: {len(arcs)} 个行为节点")

# 显示角色弧线示例
print(f"\n👤 角色弧线示例（每个角色前3个行为）:")
for char, arcs in character_arcs.items():
    print(f"   {char}:")
    for i, arc in enumerate(arcs[:3]):
        print(f"      {i+1}. {arc['chapter']}第{arc['sentence']+1}句: {arc['behavior']}")
    if len(arcs) > 3:
        print(f"      ... 还有 {len(arcs)-3} 个行为")

=== Step 6.5: 新版Behavior处理 ===
✅ 新版behavior trace生成完成
📊 角色弧线统计:
   小红帽: 68 个行为节点
   快递公司调度员: 15 个行为节点
   维修机器人“小蓝”: 23 个行为节点
   机械狼: 45 个行为节点
   空间站安保队长: 6 个行为节点
   外婆: 11 个行为节点
   空间站站长: 3 个行为节点

👤 角色弧线示例（每个角色前3个行为）:
   小红帽:
      1. Chapter 1第1句: 冷静
      2. Chapter 1第1句: 专业
      3. Chapter 1第1句: 服从指令
      ... 还有 65 个行为
   快递公司调度员:
      1. Chapter 1第1句: 冷静
      2. Chapter 1第1句: 指挥有序
      3. Chapter 1第1句: 高度信任
      ... 还有 12 个行为
   维修机器人“小蓝”:
      1. Chapter 1第5句: 积极
      2. Chapter 1第5句: 配合
      3. Chapter 1第5句: 负责
      ... 还有 20 个行为
   机械狼:
      1. Chapter 3第4句: 威胁
      2. Chapter 3第4句: 冷酷
      3. Chapter 3第4句: 控制欲强
      ... 还有 42 个行为
   空间站安保队长:
      1. Chapter 5第5句: 果断
      2. Chapter 5第5句: 权威
      3. Chapter 5第5句: 冷静
      ... 还有 3 个行为
   外婆:
      1. Chapter 6第6句: 感激
      2. Chapter 6第6句: 幸福
      3. Chapter 6第6句: 安慰
      ... 还有 8 个行为
   空间站站长:
      1. Chapter 7第6句: 赞赏
      2. Chapter 7第6句: 鼓励
      3. Chapter 7第6句: 温暖


# Step 6.7 - 联动机制

In [12]:
# Step 6.7 - 联动机制
print("=== Step 6.7: 联动机制 ===")

print(f"🔄 运行plot和dialogue联动机制...")
try:
    # 使用章节级数据进行sync
    story, chapter_results_updated, revision_log = sync_plot_and_dialogue_from_behavior(
        story, chapter_results, characters, model=behavior_model)
    
    print(f"✅ 联动机制完成")
    print(f"   修订记录数: {len(revision_log) if revision_log else 0}")
    
except Exception as e:
    print(f"⚠️ 联动机制出错: {e}")
    chapter_results_updated = chapter_results  # 添加这行
    revision_log = []

=== Step 6.7: 联动机制 ===
🔄 运行plot和dialogue联动机制...
yuanshi content: {
  "updated_plot": "小红帽身穿红色快递制服，背着装有医疗芯片的特制包裹，站在自己的快递飞船旁，通过头戴通讯器与快递公司调度员进行最后的任务确认。调度员的影像投射在飞船舱门旁的全息屏幕上，神情专注地叮嘱小红帽注意途中安全，尤其警惕近期频发的黑市机械体活动。小红帽坚定地点头，目光中透出责任感。飞船舱门缓缓关闭，动力系统启动，舱内灯光亮起，准备启程前往空间站内部，将医疗芯片安全送达外婆手中。小红帽按照调度员指令，驾驶飞船顺利抵达医疗中心，并将医疗芯片安全交付至指定人员，完成任务后返航。返航途中，小红帽与维修机器人“小蓝”取得联系，约定共同巡查空间站主控舱以确保安全，并与安保队员沟通确认巡逻路线。小蓝完成自检后与小红帽在主控舱门口集合，准备展开联合巡查。远处空间站安保队员在巡逻，整个场景充满紧张而有序的气氛。",
  "change_summary": "对话中小红帽的任务目标由原 plot 的“将医疗芯片送达外婆手中”变为“将医疗芯片交付至医疗中心指定人员”，且本章并未出现外婆。任务完成后，小红帽与小蓝共同巡查主控舱，增加了与安保队员的沟通和联合巡查情节。已据此调整 plot，使其与对话一致。",
  "changed": true
}...
yuanshi content: {
  "updated_plot": "在幽深的宇宙中,小红帽驾驶着银白色的快递飞船,悬停在一片扭曲的空间前。虫洞如同一只巨大的漩涡,闪烁着蓝紫色的电弧,吞噬着周围的星光。驾驶舱内仪表灯光闪烁,紧张的气氛弥漫。快递公司调度员的全息影像投射在控制台上,语气冷静且细致地为小红帽分析虫洞穿越的风险、注意事项,并反复叮嘱她佩戴防护装备、保持通讯、监测生命体征、校准计时器、固定快递物品等操作流程。小红帽一一确认,展现出高度的责任感和专业素养。她紧握操纵杆,目光坚定,心中既有对未知的敬畏,也有将医疗芯片安全送达外婆手中的责任感。飞船外壳反射着虫洞的光芒,舱内回响着引擎的低鸣,时间仿佛凝固在即将跃入虫洞的那一刻。",
  "change_summary": "对话内容与原 plot 基本一致，仅对调度员与小红帽的互动细节进行了补充和具体化，使其更贴合对话中反复确

# Step 7 - 保存所有输出

In [13]:
# Step 7 - 保存所有输出
print("=== Step 7: 保存所有输出 ===")

# 保存核心数据
save_json(role_state, version, "role_state.json")
save_json(story, version, "story_updated.json")
save_json(sentence_results, version, "dialogue_updated.json")  # 改为保存句子级！
save_json(revision_log, version, "revision_log.json")

print(f"💾 核心数据已保存")

# 生成小说文件
print(f"📚 生成小说文件...")

# 使用章节级编译（作为对比）
compiled_story = compile_full_story_by_chapter(story, chapter_results_updated)
save_md(compiled_story, os.path.join(folder, "novel_story_chapter.md"))
print(f"   ✅ novel_story_chapter.md 已生成（章节级）")

# 使用句子级精确编译（主要输出）
compiled_updated = compile_full_story_by_sentence(story, sentence_results)
save_md(compiled_updated, os.path.join(folder, "novel_story.md"))  # 也保存为主文件
print(f"   ✅ novel_story.md 已生成（句子级精确）")

=== Step 7: 保存所有输出 ===
💾 核心数据已保存
📚 生成小说文件...
   ✅ novel_story_chapter.md 已生成（章节级）
   ✅ novel_story.md 已生成（句子级精确）


In [14]:
# 在 Step 7 编译小说后添加
print("\n=== 验证编译结果 ===")

# 读取并检查novel_story.md
novel_path = os.path.join(folder, "novel_story.md")
if os.path.exists(novel_path):
    with open(novel_path, 'r', encoding='utf-8') as f:
        novel_content = f.read()
    
    # 检查是否有带action的对话格式
    import re
    # 匹配 "角色xxx，"对话" ——角色" 的格式
    action_pattern = r'(\w+)([^，]+)，"([^"]+)" ——\1'
    action_matches = re.findall(action_pattern, novel_content)
    
    print(f"✅ 找到 {len(action_matches)} 个带action的对话")
    if action_matches:
        print("示例：")
        for i, (speaker, action, dialogue) in enumerate(action_matches[:3]):
            print(f"   {i+1}. {speaker}{action}，\"{dialogue}\" ——{speaker}")


=== 验证编译结果 ===
✅ 找到 192 个带action的对话
示例：
   1. 小红帽身穿红色快递制服,背着装有医疗芯片的特制包裹,站在自己的快递飞船旁,通过头戴通讯器与快递公司调度员进行最后的任务确认.

小红帽调整头戴通讯器,检查包裹固定情况,等待调度员回复，"调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令." ——小红帽
   2. 快递公司调度员下达前往医疗中心的投递指令,并要求实时状态汇报.，"收到,小红帽,请确认飞船能源充足,预计航线无异常后,立即启程前往目标医疗中心,优先级为最高级别.途中保持通讯畅通,随时汇报进展." ——快递公司调度员
   3. 小红帽小红帽启动飞船,输入目标坐标,开始升空前往医疗中心.，"明白,调度员,飞船能源已满,航线一切正常,准备起飞,预计十分钟后抵达目标医疗中心." ——小红帽


# Step 8 - 增强处理

In [15]:
print("=== Step 8: 增强处理 ===")

print(f"🔄 运行故事增强...")
try:
    enhance_story_with_transitions(task_name=version, input_story_file="story_updated.json")
    print(f"   ✅ 故事过渡增强完成")
except Exception as e:
    print(f"   ⚠️ 故事增强失败: {e}")

print(f"🔄 运行对话润色...")
try:
    polish_dialogues_in_story(task_name=version, input_dialogue_file="dialogue_updated.json")
    print(f"   ✅ 对话润色完成")
except Exception as e:
    print(f"   ⚠️ 对话润色失败: {e}")

print(f"🔄 运行角色状态追踪...")
try:
    run_character_state_tracker(version=version, dialogue_file="dialogue_updated.json", model=behavior_model)
    print(f"   ✅ 角色状态追踪完成")
except Exception as e:
    print(f"   ⚠️ 角色状态追踪失败: {e}")

print(f"✅ 增强处理完成")

=== Step 8: 增强处理 ===
🔄 运行故事增强...
📝 检测到句子级对话数据，使用句子级编译...
✅ 增强版故事已生成：/Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1/enhanced_story_updated.md
   ✅ 故事过渡增强完成
🔄 运行对话润色...
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：维修机器人“小蓝” 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：维修机器人“小蓝” 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：维修机器人“小蓝” 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：维修机器人“小蓝” 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：维修机器人“小蓝” 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：小红帽 的对白
✅ 润色成功：快递公司调度员 的对白
✅ 润色成功：机械狼 的对白
✅ 润色成功：机械狼 的对白
✅ 润色成功：机械狼 的对白
✅ 润色成功：机械狼 的对白
✅ 润色成功：机械狼 的对白
✅ 润

In [16]:
# 在 Step 8 之后添加
print("\n=== 验证增强版本 ===")

enhanced_path = os.path.join(folder, "enhanced_story_dialogue_updated.md")
if os.path.exists(enhanced_path):
    with open(enhanced_path, 'r', encoding='utf-8') as f:
        enhanced_content = f.read()
    
    # 检查是否还有原始格式的对话
    original_format_count = enhanced_content.count('" ——')
    print(f"📊 增强版本中原始格式对话数: {original_format_count}")
    
    if original_format_count == 0:
        print("✅ 所有对话都已自然化！")
    else:
        print("⚠️ 还有部分对话未被润色")
    
    # 预览增强效果
    print("\n📖 增强版本预览（前800字符）：")
    print(enhanced_content[:800])


=== 验证增强版本 ===
📊 增强版本中原始格式对话数: 0
✅ 所有对话都已自然化！

📖 增强版本预览（前800字符）：
# Chapter 1：星际快递任务

小红帽身穿红色快递制服,背着装有医疗芯片的特制包裹,站在自己的快递飞船旁,通过头戴通讯器与快递公司调度员进行最后的任务确认.

小红帽调整头戴通讯器,检查包裹固定情况,等待调度员回复，小红帽站在自己的快递飞船旁，红色制服在星港灯光下格外醒目。她低头仔细检查背后的特制包裹，确认医疗芯片稳妥无损后，抬手调整了头戴通讯器的位置。耳边传来微弱的电流声，她深吸一口气，目光坚定地望向舱门前方的虚空。  
“调度员，我已抵达指定坐标。”她一边用手指轻点通讯器，一边再次确认包裹的固定装置，“医疗芯片包裹状态良好，请指示下一步投递流程。”  
说完，她静静等待着调度员的回复，指尖还在包裹扣环上轻轻摩挲，随时准备迎接新的任务。

快递公司调度员下达前往医疗中心的投递指令,并要求实时状态汇报.，调度员的声音透过头戴通讯器，清晰而冷静地传来，仿佛一股无形的力量将小红帽从脚底稳稳托起。“小红帽，出发前请再次确认飞船能源充足，检查航线是否一切正常。”他的话语一如既往地简洁有力，没有一丝多余的情绪，却让人莫名安心。“这次任务优先级最高，目标是医疗中心。请立即启程，途中保持通讯畅通，随时向我汇报进展。”  
小红帽下意识地挺直了背，手指在飞船控制面板上飞快地滑动，依照调度员的指令逐项检查。她能感受到对方那份经验丰富的镇定，仿佛无论多么紧急的任务，只要有他的调度，一切都在掌控之中。

小红帽小红帽启动飞船,输入目标坐标,开始升空前往医疗中心.，小红帽站在自己的快递飞船旁，红色制服在晨光下格外醒目。她低头检查了一遍背后的特制包裹，确认医疗芯片安然无恙。手指在控制面板上飞快地输入目标坐标，飞船舱门随即缓缓合拢。耳边传来调度员的声音，她微微一笑，语气坚定地汇报：“飞船能源已满，航线一切正常，准备起飞。”随着引擎启动的低鸣，她调整好头戴通讯器，目光专注地望向前方星空，“预计十


In [20]:
def debug_action_flow(version):
    """调试action信息的完整流程"""
    print("🔍 调试ACTION信息流程")
    
    folder = os.path.join(output_dir, version)
    
    # 1. 检查dialogue_updated.json
    dialogue_path = os.path.join(folder, "dialogue_updated.json")
    if os.path.exists(dialogue_path):
        dialogue_data = load_json(dialogue_path)
        
        action_count = 0
        action_examples = []
        
        for sentence in dialogue_data:
            if sentence.get("dialogue"):
                for d in sentence["dialogue"]:
                    if d.get("action"):
                        action_count += 1
                        if len(action_examples) < 3:
                            action_examples.append({
                                "chapter": sentence["chapter_id"],
                                "sentence": sentence["sentence_index"],
                                "speaker": d["speaker"],
                                "action": d["action"],
                                "dialogue": d["dialogue"]
                            })
        
        print(f"\n📊 Action统计：")
        print(f"   总action数: {action_count}")
        print(f"\n🎯 Action示例：")
        for ex in action_examples:
            print(f"   {ex['chapter']} 句{ex['sentence']+1}:")
            print(f"   {ex['speaker']}: {ex['action']}")
            print(f'   对话: "{ex["dialogue"]}"')  # 修复：使用单引号包裹，内部用双引号
            print()
    
    # 2. 检查novel_story.md中的action
    novel_path = os.path.join(folder, "novel_story.md")
    if os.path.exists(novel_path):
        with open(novel_path, 'r', encoding='utf-8') as f:
            novel = f.read()
        
        # 查找带逗号的格式
        import re
        pattern = r'([^，\n]+)，"([^"]+)" ——(\w+)'
        matches = re.findall(pattern, novel)
        
        print(f"\n📖 小说中的Action格式：")
        print(f"   找到 {len(matches)} 个可能的action格式")
        if matches:
            print(f"\n示例：")
            for i, (action_part, dialogue, speaker) in enumerate(matches[:3]):
                if speaker in action_part:
                    print(f'   {i+1}. {action_part}，"{dialogue}" ——{speaker}')  # 修复
    
    # 3. 检查enhanced版本
    enhanced_path = os.path.join(folder, "enhanced_story_dialogue_updated.md")
    if os.path.exists(enhanced_path):
        print(f"\n📚 增强版本存在：{enhanced_path}")
        print(f"   可以检查润色效果")
    
    return action_count > 0

# 运行调试
if debug_action_flow(version):
    print("\n✅ Action信息流程正常！")
else:
    print("\n❌ 未检测到Action信息")

🔍 调试ACTION信息流程

📊 Action统计：
   总action数: 192

🎯 Action示例：
   Chapter 1 句1:
   小红帽: 调整头戴通讯器,检查包裹固定情况,等待调度员回复
   对话: "调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令."

   Chapter 1 句1:
   快递公司调度员: 下达前往医疗中心的投递指令,并要求实时状态汇报.
   对话: "收到,小红帽,请确认飞船能源充足,预计航线无异常后,立即启程前往目标医疗中心,优先级为最高级别.途中保持通讯畅通,随时汇报进展."

   Chapter 1 句1:
   小红帽: 小红帽启动飞船,输入目标坐标,开始升空前往医疗中心.
   对话: "明白,调度员,飞船能源已满,航线一切正常,准备起飞,预计十分钟后抵达目标医疗中心."


📖 小说中的Action格式：
   找到 192 个可能的action格式

示例：
   1. 小红帽调整头戴通讯器,检查包裹固定情况,等待调度员回复，"调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令." ——小红帽
   2. 快递公司调度员下达前往医疗中心的投递指令,并要求实时状态汇报.，"收到,小红帽,请确认飞船能源充足,预计航线无异常后,立即启程前往目标医疗中心,优先级为最高级别.途中保持通讯畅通,随时汇报进展." ——快递公司调度员
   3. 小红帽小红帽启动飞船,输入目标坐标,开始升空前往医疗中心.，"明白,调度员,飞船能源已满,航线一切正常,准备起飞,预计十分钟后抵达目标医疗中心." ——小红帽

📚 增强版本存在：/Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1/enhanced_story_dialogue_updated.md
   可以检查润色效果

✅ Action信息流程正常！


# 最终结果检查

In [18]:
print("=== 最终结果检查 ===")

# 检查生成的文件
output_files = [
    "story.json", "characters.json", "dialogue_marks.json",
    "sentence_analysis.json", "behavior_timeline_raw.json", "behavior_trace.json",
    "story_updated.json", "dialogue_updated.json", "role_state.json",
    "novel_story.md", "novel_story_updated.md"
]

print(f"📁 检查输出文件:")
for filename in output_files:
    filepath = os.path.join(folder, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        print(f"   ✅ {filename}: {size} bytes")
    else:
        print(f"   ❌ {filename}: 缺失")

# 显示关键统计
print(f"\n📊 最终统计:")
print(f"   版本名称: {version}")
print(f"   输出目录: {folder}")
print(f"   故事章节数: {len(story)}")
print(f"   角色数量: {len(characters)}")
print(f"   句子分析数量: {len(sentence_results)}")
print(f"   行为记录数量: {len(behavior_timeline)}")

print(f"\n🎉 完整流程执行完毕！")
print(f"📂 所有文件保存在: {folder}")

# 显示生成的小说预览
try:
    with open(os.path.join(folder, "novel_story.md"), 'r', encoding='utf-8') as f:
        novel_content = f.read()
    print(f"\n📖 生成小说预览（前500字符）:")
    print(f"{novel_content[:500]}...")
except:
    print(f"❌ 无法读取生成的小说文件")

=== 最终结果检查 ===
📁 检查输出文件:
   ✅ story.json: 8141 bytes
   ✅ characters.json: 2847 bytes
   ✅ dialogue_marks.json: 72241 bytes
   ❌ sentence_analysis.json: 缺失
   ✅ behavior_timeline_raw.json: 108094 bytes
   ✅ behavior_trace.json: 164194 bytes
   ✅ story_updated.json: 12703 bytes
   ✅ dialogue_updated.json: 92643 bytes
   ✅ role_state.json: 5976 bytes
   ✅ novel_story.md: 55267 bytes
   ❌ novel_story_updated.md: 缺失

📊 最终统计:
   版本名称: 小红帽_科幻_linear_T0.7_s1
   输出目录: /Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1
   故事章节数: 7
   角色数量: 8
   句子分析数量: 42
   行为记录数量: 171

🎉 完整流程执行完毕！
📂 所有文件保存在: /Users/haha/Story/data/output/小红帽_科幻_linear_T0.7_s1

📖 生成小说预览（前500字符）:
# Chapter 1：星际快递任务

小红帽身穿红色快递制服,背着装有医疗芯片的特制包裹,站在自己的快递飞船旁,通过头戴通讯器与快递公司调度员进行最后的任务确认.

小红帽调整头戴通讯器,检查包裹固定情况,等待调度员回复，"调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令." ——小红帽

快递公司调度员下达前往医疗中心的投递指令,并要求实时状态汇报.，"收到,小红帽,请确认飞船能源充足,预计航线无异常后,立即启程前往目标医疗中心,优先级为最高级别.途中保持通讯畅通,随时汇报进展." ——快递公司调度员

小红帽小红帽启动飞船,输入目标坐标,开始升空前往医疗中心.，"明白,调度员,飞船能源已满,航线一切正常,准备起飞,预计十分钟后抵达目标医

In [19]:
# 在你的notebook中运行这个调试代码

print("🔍 调试sentence_results数据...")

# 1. 检查sentence_results的结构
if 'sentence_results' in locals() and sentence_results:
    print(f"✅ sentence_results存在，数量: {len(sentence_results)}")
    
    # 检查前几个句子的结构
    print(f"\n📊 前3个句子的结构:")
    for i, sentence in enumerate(sentence_results[:3]):
        print(f"句子{i+1}:")
        print(f"  章节: {sentence.get('chapter_id', 'Unknown')}")
        print(f"  句子索引: {sentence.get('sentence_index', 'Unknown')}")
        print(f"  需要对话: {sentence.get('need_to_action', 0)}")
        print(f"  演员: {sentence.get('actor_list', [])}")
        print(f"  有dialogue字段: {'dialogue' in sentence}")
        
        if sentence.get('dialogue'):
            print(f"  对话数量: {len(sentence['dialogue'])}")
            if sentence['dialogue']:
                print(f"  第一个对话: {sentence['dialogue'][0]}")
        else:
            print(f"  对话: 空")
        print()
    
    # 统计有对话的句子
    dialogue_sentences = [s for s in sentence_results if s.get('need_to_action') == 1 and s.get('dialogue')]
    print(f"📈 统计:")
    print(f"  总句子数: {len(sentence_results)}")
    print(f"  需要对话的句子: {len([s for s in sentence_results if s.get('need_to_action') == 1])}")
    print(f"  实际有对话数据的句子: {len(dialogue_sentences)}")
    
    if dialogue_sentences:
        print(f"\n✅ 有对话的句子示例:")
        sample = dialogue_sentences[0]
        print(f"  章节: {sample['chapter_id']}")
        print(f"  句子索引: {sample['sentence_index']}")
        print(f"  句子内容: {sample['sentence'][:50]}...")
        print(f"  对话数量: {len(sample['dialogue'])}")
        print(f"  对话示例: {sample['dialogue'][0] if sample['dialogue'] else 'None'}")
    else:
        print(f"❌ 没有找到有对话数据的句子！")
        
else:
    print(f"❌ sentence_results不存在或为空")

# 2. 检查compile_full_story_by_sentence的映射逻辑
print(f"\n🔍 测试对话映射逻辑...")

if 'sentence_results' in locals() and sentence_results:
    # 模拟compile_full_story_by_sentence的映射逻辑
    dialogue_map = {}
    for item in sentence_results:
        if item.get("need_to_action") == 1 and item.get("dialogue"):
            chapter_id = item["chapter_id"]
            sentence_idx = item["sentence_index"]
            
            if chapter_id not in dialogue_map:
                dialogue_map[chapter_id] = {}
            dialogue_map[chapter_id][sentence_idx] = item["dialogue"]
    
    print(f"📋 对话映射结果:")
    for chapter_id, chapter_dialogues in dialogue_map.items():
        print(f"  {chapter_id}: {len(chapter_dialogues)} 个句子有对话")
        for sent_idx, dialogues in list(chapter_dialogues.items())[:2]:  # 只显示前2个
            print(f"    句子{sent_idx}: {len(dialogues)} 条对话")
    
    if not dialogue_map:
        print(f"❌ 对话映射为空！这就是问题所在")
    else:
        print(f"✅ 对话映射正常")

🔍 调试sentence_results数据...
✅ sentence_results存在，数量: 42

📊 前3个句子的结构:
句子1:
  章节: Chapter 1
  句子索引: 0
  需要对话: 1
  演员: ['小红帽', '快递公司调度员']
  有dialogue字段: True
  对话数量: 11
  第一个对话: {'speaker': '小红帽', 'dialogue': '调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令.', 'action': '调整头戴通讯器,检查包裹固定情况,等待调度员回复'}

句子2:
  章节: Chapter 1
  句子索引: 1
  需要对话: 1
  演员: ['快递公司调度员', '小红帽']
  有dialogue字段: True
  对话数量: 11
  第一个对话: {'speaker': '快递公司调度员', 'dialogue': '小红帽,出发前请再次确认你的货物安全锁定,途中务必保持警惕,最近黑市机械体活动频繁,遇到异常情况立即汇报.', 'action': '全息屏幕上的影像微微闪烁,调度员目光严肃地看向小红帽,语气中带着关切和警觉.'}

句子3:
  章节: Chapter 1
  句子索引: 2
  需要对话: 0
  演员: ['小红帽']
  有dialogue字段: True
  对话: 空

📈 统计:
  总句子数: 42
  需要对话的句子: 19
  实际有对话数据的句子: 19

✅ 有对话的句子示例:
  章节: Chapter 1
  句子索引: 0
  句子内容: 小红帽身穿红色快递制服,背着装有医疗芯片的特制包裹,站在自己的快递飞船旁,正通过头戴通讯器与快递公司...
  对话数量: 11
  对话示例: {'speaker': '小红帽', 'dialogue': '调度员,我已经到达指定坐标,医疗芯片包裹状态良好,请确认下一步投递指令.', 'action': '调整头戴通讯器,检查包裹固定情况,等待调度员回复'}

🔍 测试对话映射逻辑...
📋 对话映射结果:
  Chapter 1: 3 个句子有对话
    句子0: 11 条对话
    句子1: 11 条对话
  Chapter 2: 1 个句子有对话
    

In [ ]:

# ===============================================
# 方案1: 重新分配对话到句子级（在notebook中运行）
# ===============================================

def redistribute_dialogues_to_sentences(story, chapter_results, sentence_results):
    """
    将章节级对话重新分配到句子级
    """
    print("🔄 重新分配对话到句子级...")
    
    # 按章节分组
    chapters_sentences = {}
    for sentence in sentence_results:
        chapter_id = sentence['chapter_id']
        if chapter_id not in chapters_sentences:
            chapters_sentences[chapter_id] = []
        chapters_sentences[chapter_id].append(sentence)
    
    # 重新分配对话
    fixed_sentence_results = []
    
    for chapter_idx, chapter_result in enumerate(chapter_results):
        chapter_id = story[chapter_idx].get('chapter_id', f'Chapter {chapter_idx+1}')
        chapter_dialogues = chapter_result.get('dialogue', [])
        
        # 获取这个章节的句子
        chapter_sentences = chapters_sentences.get(chapter_id, [])
        need_dialogue_sentences = [s for s in chapter_sentences if s.get('need_to_action') == 1]
        
        print(f"  {chapter_id}: {len(chapter_dialogues)} 条对话，{len(need_dialogue_sentences)} 个句子需要对话")
        
        # 分配对话到句子
        if chapter_dialogues and need_dialogue_sentences:
            dialogues_per_sentence = len(chapter_dialogues) // len(need_dialogue_sentences)
            remaining_dialogues = len(chapter_dialogues) % len(need_dialogue_sentences)
            
            dialogue_idx = 0
            for sentence in chapter_sentences:
                if sentence.get('need_to_action') == 1 and dialogue_idx < len(chapter_dialogues):
                    # 分配对话
                    num_dialogues = dialogues_per_sentence
                    if remaining_dialogues > 0:
                        num_dialogues += 1
                        remaining_dialogues -= 1
                    
                    sentence_dialogues = chapter_dialogues[dialogue_idx:dialogue_idx + num_dialogues]
                    sentence['dialogue'] = sentence_dialogues
                    dialogue_idx += num_dialogues
                    
                    print(f"    句子{sentence['sentence_index']}: 分配了 {len(sentence_dialogues)} 条对话")
                else:
                    sentence['dialogue'] = []
                
                fixed_sentence_results.append(sentence)
        else:
            # 没有对话的章节
            for sentence in chapter_sentences:
                sentence['dialogue'] = []
                fixed_sentence_results.append(sentence)
    
    return fixed_sentence_results

# 运行重新分配
if 'sentence_results' in locals() and 'chapter_results' in locals():
    print("🔧 开始重新分配对话...")
    fixed_sentence_results = redistribute_dialogues_to_sentences(story, chapter_results, sentence_results)
    
    # 验证结果
    dialogue_count = len([s for s in fixed_sentence_results if s.get('dialogue')])
    print(f"✅ 重新分配完成，{dialogue_count} 个句子有对话数据")
    
    # 保存修正后的数据
    save_json(fixed_sentence_results, version, "sentence_dialogues_fixed.json")
    
    # 重新编译小说
    print("📖 使用修正后的数据重新编译小说...")
    compiled_fixed = compile_full_story_by_sentence(story, fixed_sentence_results)
    save_md(compiled_fixed, os.path.join(folder, "novel_story_SENTENCE_FIXED.md"))
    
    # 检查结果
    dialogue_in_novel = compiled_fixed.count('" ——')
    print(f"✅ 修正版小说生成完成，检测到 {dialogue_in_novel} 条对话")
    
    if dialogue_in_novel > 0:
        print("🎉 成功！对话已按句子精确插入")
        # 预览
        print(f"\n📖 预览（前800字符）:")
        print(compiled_fixed[:800])
    else:
        print("❌ 仍然没有对话，需要进一步调试")
